In [ ]:
import os
from arc_prize.synth_data.prompt import get_html, make_prompt

puzzle_id = "1d398264"

print(make_prompt(puzzle_id))

# for model in ["claude"]:
#   for i in range(1):
#     response = get_html(puzzle_id, model, output_file_path=f"html/{puzzle_id}/{model}_{i}.html")


In [ ]:
import json
import os
import time
from arc_prize.synth_data.html import capture_html_screenshot, get_web_driver, process_screenshot

num_puzzles = 9000
dataset_dir = "/Users/pfh/work/arc-data/html"
edition = 1

os.makedirs(dataset_dir, exist_ok=True)

driver = get_web_driver(60, 150)

# puzzles = valid_puzzles
puzzles = ["5ffb2104"]

def validate_puzzle(puzzle: list[list[int]]) -> bool:
    if len(puzzle) < 2 or len(puzzle) > 5:
        return False
    for pair in puzzle:
        if len(pair) != 2:
            return False
    return True

for puzzle_id in puzzles:
    html_file = f"html/{puzzle_id}.html"
    current_dir = os.getcwd()
    full_path = os.path.join(current_dir, html_file)
    full_path = os.path.abspath(full_path)

    start_time = time.time()
    puzzles = []

    print("Starting", puzzle_id)

    for i in range(num_puzzles):
        try:
            raw_screenshot = capture_html_screenshot(driver, full_path)
            arc_puzzle_data, _ = process_screenshot(raw_screenshot)
        except Exception as e:
            print("Skipping", e)
            continue

        if validate_puzzle(arc_puzzle_data) is True:
            puzzles.append(arc_puzzle_data)

        if i % 100 == 0:
            duration = time.time() - start_time
            per_puzzle_time = duration / max(len(puzzles), 1)
            print(
                f"Iteration {i}, done {len(puzzles)} puzzles. Time elapsed: {duration:.2f}s ({(duration / 60):.2f}m). Per puzzle: {per_puzzle_time:.2f}s"
            )
    
    duration = time.time() - start_time
    print("Finished", puzzle_id, f"Total time: {duration:.2f}s ({(duration / 60):.2f}m)")

    with open(f"{dataset_dir}/{puzzle_id}_{edition}.json", "w") as f:
        json.dump(puzzles, f)

driver.quit()
